In [8]:
import pymongo
import warnings
import pandas as pd
import sqlite3
import json
from datetime import datetime
import openpyxl

try:

    # Global Variables 
    fieldList = []
    cacheTabList = []
    resultsData = []
    cacheKeysList = []
    queryResult = []
    projectList = {}
    fileName = "inputFile.xlsx"
    outputFileName = "outputFile.xlsx"
    ## Cache Handling 
    def addCacheWhereKeys(attribute, value,userCondition, cacheKeysList):
        if isinstance(value,str):  
            if userCondition == "Like":
                value = str("'%" + value + "%'")
                cacheKeysList.append(attribute + " like " + value)
            elif userCondition == "Equal":
                value = str("'" + value + "'")
                cacheKeysList.append(attribute + " = " + value)
        elif isinstance(value,float) and pd.isna(value) == False:
            if userCondition == "Equal":
                value = str(value)
                cacheKeysList.append(attribute + " = " + value)
            else:
                value = str(value)
                cacheKeysList.append(attribute + " " + userCondition + " " + value)      
        elif isinstance(value,int):
            if userCondition == "Equal":
                value = str(value)
                cacheKeysList.append(attribute + " = " + value)
            else:
                value = str(value)
                cacheKeysList.append(attribute + " " + userCondition + " " + value)

        return cacheKeysList
    # Connection to CACHE DB
    def connect_sql_database():
        sqlDB = {}
        conn = sqlite3.connect('Cache_AirBnB.db') 
        cur = conn.cursor()
        print("***Cache DB Connection established***")
        sqlDB["connection"] = conn
        sqlDB["cursor"] = cur
        return sqlDB

    ## Get Cached Results from the CACHE DB
    def get_cache_data(cur,cacheKeysList,logicalOp ):  
        global fieldList
        cacheResult = "Hit"
        tabName = ""
        cacheKeys = ""
        fieldNames = ""
        fieldNameLists = []
        fieldTypeLists = []
        insertValuesList = []
        insertFieldList = []
        # Parse the field Names 
        for i in range(len(cacheTabList)):
                cacheTabDict = cacheTabList[i]
                fieldName = list(cacheTabDict.keys())[0]
                fieldType = list(cacheTabDict.values())[0]
                tabName = tabName + fieldName[0:3]
                fieldNames = fieldNames + fieldName + " " + fieldType + ", "
                fieldNameLists.append(fieldName)
                fieldTypeLists.append(fieldType)
        #Remove the last string
        fieldNames = fieldNames[0:len(fieldNames)-2]

        print("***Checking query in Cache DB***")
        selectStatement = "select count(*) from Cache_LookUp where tablename = '" + tabName + "'"
        cur = sqlDB["cursor"] 
        conn =  sqlDB["connection"] 
        cursor = cur.execute(selectStatement)
       
        for row in cursor:
            lookUpCount_str = row[0]
        lookUpCount = int(lookUpCount_str)
        
        if int(lookUpCount) > 0:
            now = datetime.now()
            lastAccessed = now.strftime("%d%m%Y%H%M%S")
            print("***Cache DB: Look up table Key exist*** ")    
            fieldNameLists = ','.join(fieldNameLists)
            cacheKeys = logicalOp.join(cacheKeysList)
            selectStatement = "select " + fieldNameLists + " from " + tabName + " where ( " + cacheKeys + " )"
            print(selectStatement)
            cursor = cur.execute(selectStatement)
            for row in cursor:
                queryResult.append(row)
                
            # Update the last accessed to the Cache table 
            updateStatement = "update  Cache_LookUp  set LastAccessed =  '" + lastAccessed + "'  where TableName='" + tabName + "'"
            cursor = cur.execute(updateStatement)
            conn.commit()
            # Format the results
            formatResults(queryResult,fieldNameLists)
           
            
        else:
            cacheResult = "Miss"

        return cacheResult


    ## Excel Reading for Input
    def readInputExcel(fileName):
        warnings.filterwarnings('ignore',category=UserWarning, module='openpyxl')
        userData = pd.read_excel(fileName, sheet_name="Input", usecols = "A,B,C,D,E")
        return userData
    
    
    ## Excel Reading for MONGO DB Connection Parameters 
    def readMongoConnectExcel(fileName):
        warnings.filterwarnings('ignore',category=UserWarning, module='openpyxl')
        conMongoDBData = pd.read_excel(fileName, sheet_name="MongoDBConnect", usecols = "A,B,C")
        connectMongoDB = conMongoDBData[['Username', 'Password','QueryLimit']]
        return connectMongoDB


    ## MONGO DB Query Building
    def addAttributes(fieldList, key, value, userCondition):
        global cacheTabList
        if isinstance(value,str):
            if userCondition == "Like":
                condDict = {"$regex":value}
                fieldDict = {key:condDict}
            else:
                fieldDict = {key:value}
            fieldList = fieldList.append(fieldDict)
            cacheTab = {key:"TEXT"}
            cacheTabList.append(cacheTab)

        elif isinstance(value,float) and pd.isna(value) == False:
            fieldDict = {key:value}
            fieldList.append(fieldDict)
            cacheTab = {key:"FLOAT"}
            cacheTabList.append(cacheTab)
        elif isinstance(value,int):
            if userCondition == '>=':
                condDict = {"$gte":value}
                fieldDict = {key:condDict}
            elif userCondition == '<=':
                condDict = {"$lte":value}
                fieldDict = {key:condDict}
            elif userCondition == 'Equal':
                condDict = {"$eq":value}
                fieldDict = {key:condDict}
            else:
                fieldDict = {key:value}
            fieldList = fieldList.append(fieldDict)
            cacheTab = {key:"INTEGER"}
            cacheTabList.append(cacheTab)


    ## MONGO DB Projection Handling
    def addProjectionFields(projectList,attribute,projectField,index):
        if len(projectList) == 0:
            projectList.update({"_id": 1})
        if projectField == 'X':
            projectList.update({attribute : index+2 })
        


    ## MONGO DB Connection
    def connect_mongo(connectMongoDB):
        username  = connectMongoDB.to_dict()['Username'][0]
        password = connectMongoDB.to_dict()['Password'][0]
        QueryLimit = connectMongoDB.to_dict()['QueryLimit'][0]
        url = "mongodb+srv://" + username + ":" + password + "@cluster0.0nthl.mongodb.net/sample_airbnb?retryWrites=true&w=majority"
        myConnect = pymongo.MongoClient(url)
        db = myConnect["sample_airbnb"]
        print("***MongoDB: Connection Established***")
        collection = db["listingsAndReviews"]
        return collection

    ## MONGO DB Query Execution
    def performQuery(collection,fieldList,projectList,logicalOp):
        pipeline = [
            {"$match": {
                        logicalOp: fieldList
                        }},

            { "$project": projectList},
            { "$limit" : 10 }
        ] 
        results = collection.aggregate(pipeline)
        print("***MongoDB: Query Executed***")
        for record in results:
            resultsData.append(record)
        return resultsData

    ## Get the result Count
    def getResultCount(results):
        count = 0
        for record in results: 
            count = count + 1
        return count


    ## Display Final Results
    def formatResults(results,fieldNameLists):
        # create DataFrame using data
        my_list = fieldNameLists.split(",")
        df = pd.DataFrame(results, columns = my_list)
        print(df)
        df.to_excel(outputFileName, sheet_name='Results')
        
    
    

    ## Save to Cache
    def saveToCache(cur, results):
        tabName = ""
        fieldNames = ""
        rowCount = 0
        fieldNameLists = []
        fieldTypeLists = []
        insertValuesList = []
        insertFieldList = []
        # datetime object containing current date and time
        now = datetime.now()
        dt_string = now.strftime("%d%m%Y%H%M%S")
        lastAccessed = dt_string
        # Parse the field Names 
        for i in range(len(cacheTabList)):
                cacheTabDict = cacheTabList[i]
                fieldName = list(cacheTabDict.keys())[0]
                fieldType = list(cacheTabDict.values())[0]
                tabName = tabName + fieldName[0:3]
                fieldNames = fieldNames + fieldName + " " + fieldType + ", "
                fieldNameLists.append(fieldName)
                fieldTypeLists.append(fieldType)
        #Remove the last string
        fieldNames = fieldNames[0:len(fieldNames)-2]

        print("***Cache DB: Saving results ***") 
        selectStatement = "select count(*) from Cache_LookUp"
        cur = sqlDB["cursor"] 
        conn = sqlDB["connection"] 
        cursor = cur.execute(selectStatement)
        for row in cursor:
            rowCount_str = row[0]
        rowCount = int(rowCount_str)
        print("***Cache DB: Cache Row count :" + str(rowCount) + "*** ") 
        if int(rowCount) >= 5:
            print("***Cache DB: Checking LRU..*** ") 
            selectStatement = "select min(LastAccessed), TableName from Cache_LookUp"
            cursor = cur.execute(selectStatement)
            
            for row in cursor:
                print("***Cache DB: LRU -> Table to be dropped: "+row[1])
                tabname = row[1]
            # Delete the record from the cache look up table 
            deleteStatement = "delete from Cache_LookUp where TableName = '" + tabname + "'"
            cursor = cur.execute(deleteStatement)
            
            # Drop the table 
            dropStatement = "drop table '" + tabname + "'"
            cur.execute(deleteStatement)
            conn.commit()
            
        # Seq. No Count
        rowCount = rowCount + 1
              
    # Insert a record in the Look up table 
        rowcount = getResultCount(results)
        conn = sqlDB["connection"] 
        cur = sqlDB["cursor"] 
        insertStatement = "INSERT INTO Cache_LookUp (SNo, TableName,RowCount,LastAccessed) VALUES (" + str(rowCount) + ",'" + tabName + "',"+ str(rowcount) + "," + str(lastAccessed) + ")"
        cur.execute(insertStatement)
        print("***Cache DB: Look up table Key inserted*** ")   
# Create the temp table for caching the results 
        createStatement = "CREATE TABLE " + tabName + " ( " + fieldNames + " )"
        cur.execute(createStatement)
        print("***Cache DB: Cache Table created successfully*** ")   
        for i in range(len(results)):
            for j in range(len(cacheTabList)):
                cacheTabDict = cacheTabList[j]
                fieldName = list(cacheTabDict.keys())[0]
                fieldType = list(cacheTabDict.values())[0]

                if fieldType == 'TEXT':
                    value = str(results[i][fieldName])
                    # Escape characters for single quotation 
                    value = value.replace("'", "''")
                    value = "'" + value + "'"
                else: 
                    value = str(results[i][fieldName])
                insertFieldList.append(fieldName)
                insertValuesList.append(value)
            insertValues = ','.join(insertValuesList)
            insertField = ','.join(insertFieldList)
            insertResultStatement = "INSERT INTO " +  tabName + "( " + insertField + ") VALUES (" + insertValues +  ")"
            insertValuesList = []
            insertFieldList = []
            cur.execute(insertResultStatement)
        conn.commit()


    # Get the User input data from the excel sheet 
    userData = readInputExcel(fileName)
    userFieldList = userData["Field"]
    userValueList = userData["Value"]
    userConditionList = userData["Condition"]
    userProjectList = userData["Projection"]
    logicalOp = userData["LogicalOperator"] 
    
    
    # Get the MONGO DB Connection details from the excel sheet 
    connectMongoDB = readMongoConnectExcel(fileName)
    
    # Convert the User List to Query formats
    for index in range(len(userFieldList)):
        addCacheWhereKeys(userFieldList[index],userValueList[index],userConditionList[index],cacheKeysList) 
        addAttributes(fieldList,userFieldList[index],userValueList[index],userConditionList[index])
        addProjectionFields(projectList,userFieldList[index], userProjectList[index], index)


    # Connect to Cache DB
    sqlDB = connect_sql_database()
    cacheResult = get_cache_data(sqlDB,cacheKeysList,' ' + logicalOp[0] + ' ')
    
    # If Cache Result is Miss then connect to Mongo DB
    if cacheResult == "Miss":
        # Connect to Mongo DB
        collection = connect_mongo(connectMongoDB)
        results = performQuery(collection,fieldList,projectList,'$'+logicalOp[0])
        saveToCache(sqlDB, results)
        queryResult = get_cache_data(sqlDB,cacheKeysList,' ' + logicalOp[0] + ' ')
    
    
        
except ValueError as ValueError:
    print(str(ValueError))


{'_id': 1, 'name': 2}
{'_id': 1, 'name': 2, 'property_type': 3}
{'_id': 1, 'name': 2, 'property_type': 3}
{'_id': 1, 'name': 2, 'property_type': 3, 'accommodates': 5}
{'_id': 1, 'name': 2, 'property_type': 3, 'accommodates': 5}
***Cache DB Connection established***
***Checking query in Cache DB***
***Cache DB: Look up table Key exist*** 
select property_type,accommodates from proacc where ( property_type = 'Apartment' and accommodates <= 4 )
  property_type  accommodates
0     Apartment             2
1     Apartment             2
2     Apartment             2
3     Apartment             2
4     Apartment             2
5     Apartment             1
6     Apartment             2
7     Apartment             4
8     Apartment             2
9     Apartment             3
